In [15]:
import paho.mqtt.client as mqtt
import json
import time

In [16]:
import paho.mqtt.client as mqtt
import json
import time

MQTT_BROKER = "mqtt-dashboard.com"
MQTT_PORT = 1883
MQTT_TOPIC_PID = "robot/set/pid"
MQTT_TOPIC_MOTORS = "robot/set/motors"
MQTT_TOPIC_LOG = "robot/pid/log"
MQTT_TOPIC_RESPONSE = "robot/request"

In [17]:
mqtt_client = mqtt.Client("PID ctl")

logs = []

experiment_ongoing = False

In [18]:
def send_mqtt_message(topic, message):
    mqtt_client.publish(topic, json.dumps(message))

In [19]:
def run_experiment(Kp, Ti, Td):
    global logs
    global mqtt_client
    global experiment_ongoing
    mqtt_client = mqtt.Client("PID ctl")
    mqtt_client.connect(MQTT_BROKER, MQTT_PORT)
    mqtt_client.subscribe(MQTT_TOPIC_LOG)
    mqtt_client.on_message = on_log_message

    mqtt_client.loop()
    logs = []

    pid_settings = {"Kp": Kp, "Ti": Ti, "Td": Td}
    send_mqtt_message(MQTT_TOPIC_PID, pid_settings)

    motors_speed = {"left": 50, "right": 50}
    send_mqtt_message(MQTT_TOPIC_MOTORS, motors_speed)
    experiment_ongoing = True
    start_time = time.time()
    while time.time() - start_time < 4:
        mqtt_client.loop()

    motors_speed = {"left": 0, "right": 0}
    send_mqtt_message(MQTT_TOPIC_MOTORS, motors_speed)

    while time.time() - start_time < 4:
        mqtt_client.loop()
    
    send_mqtt_message(MQTT_TOPIC_RESPONSE, "")
    experiment_ongoing = False
    mqtt_client.disconnect()

def on_log_message(client, userdata, message):
    global experiment_ongoing
    log_data = json.loads(message.payload.decode())
    if len(logs) != 0: 
        logs.append(log_data) 
    elif len(logs) == 0 and log_data["id"] == 0:
        logs.append(log_data)

In [20]:
mqtt_client.subscribe(MQTT_TOPIC_LOG)
mqtt_client.on_message = on_log_message

In [21]:
import math
def calc_error(Kp, Ti, Td):
    global logs

    absolute_errors = []
    relative_errors = []
    settling_times = []

    sampling_time = 5

    run_experiment(Kp, Ti, Td)

    for i in range(len(logs)):
        measurement = logs[i]
        absolute_error = abs(measurement['target'] - measurement['actual'])
        relative_error = (absolute_error / measurement['target']) * 100 if measurement['target'] != 0 else 0
        absolute_errors.append(absolute_error)
        relative_errors.append(relative_error)
        
        # Sprawdzenie, czy wartość docelowa została osiągnięta
        if relative_error < 2:  # Możesz dostosować wartość graniczną według potrzeb
            settling_time = i * sampling_time
            settling_times.append(settling_time)

    mean_error = sum(absolute_errors) / len(absolute_errors)

    mean_relative_error = sum(relative_errors) / len(relative_errors)

    rmse_error = math.sqrt(sum([(error ** 2) for error in absolute_errors]) / len(absolute_errors))

    average_settling_time = sum(settling_times) / len(settling_times) if settling_times else None

    # Wyświetlenie wyników
    print("Mean error: ", mean_error)
    print("Mean relative error: ", mean_relative_error)
    print("(RMSE): ", rmse_error)
    print("Average settling time: ", average_settling_time, " ms")

    print(logs)

In [25]:
PID_settings = [
    { "Kp" : 11.98725113, "Ti" : 1.59488759, "Td" : 1.34889701 },
    { "Kp" : 29.50415103, "Ti" : 14.54508991, "Td" : 1.37240407 },
    { "Kp" : 6.48107301, "Ti" : 2.93091131, "Td" : 2.34173984 },
    { "Kp" : 12.48496515, "Ti" : 22.12299436, "Td" : 24.81674137 }
]

# for setting in PID_settings:
#     print(setting)
#     calc_error(setting["Kp"], setting["Ti"], setting["Td"])

In [31]:
setting = PID_settings[2]
calc_error(setting["Kp"], setting["Ti"], setting["Td"])

Mean error:  0.9762480857580395
Mean relative error:  11.691593841413663
(RMSE):  1.7782670901183135
Average settling time:  1490.0  ms
[{'id': 0, 'target': 8.35, 'actual': 0.0}, {'id': 1, 'target': 8.35, 'actual': 0.0}, {'id': 2, 'target': 8.35, 'actual': 0.0}, {'id': 3, 'target': 8.35, 'actual': 0.0}, {'id': 4, 'target': 8.35, 'actual': 0.2}, {'id': 5, 'target': 8.35, 'actual': 0.14}, {'id': 6, 'target': 8.35, 'actual': 0.29}, {'id': 7, 'target': 8.35, 'actual': 0.6}, {'id': 8, 'target': 8.35, 'actual': 0.61}, {'id': 9, 'target': 8.35, 'actual': 0.82}, {'id': 10, 'target': 8.35, 'actual': 0.97}, {'id': 11, 'target': 8.35, 'actual': 1.27}, {'id': 12, 'target': 8.35, 'actual': 1.28}, {'id': 13, 'target': 8.35, 'actual': 1.48}, {'id': 14, 'target': 8.35, 'actual': 1.63}, {'id': 15, 'target': 8.35, 'actual': 1.73}, {'id': 16, 'target': 8.35, 'actual': 1.8}, {'id': 17, 'target': 8.35, 'actual': 2.04}, {'id': 18, 'target': 8.35, 'actual': 2.02}, {'id': 19, 'target': 8.35, 'actual': 2.0}, {

In [10]:
# setting = PID_settings[1]
# calc_error(setting["Kp"], setting["Ti"], setting["Td"])

In [11]:
# import pandas as pd
# import plotly
# import plotly.graph_objs as go


# #Read cars data from csv
# data = {"KP" : (1, 2, 3), "TI" : (2, 3, 4), "TD" : (3, 4, 5) }


# #Make Plotly figure
# fig1 = go.Scatter3d(x=data["KP"],
#                     y=data["TI"],
#                     z=data["TD"],
#                     marker=dict(color=13,
#                                 opacity=1,
#                                 reversescale=True,
#                                 colorscale='Blues',
#                                 size=5),
#                     line=dict (width=0.02),
#                     mode='markers')

# #Make Plot.ly Layout
# mylayout = go.Layout(scene=dict(xaxis=dict( title="KP"),
#                                 yaxis=dict( title="TI"),
#                                 zaxis=dict(title="TD")),)

# #Plot and save html
# plotly.offline.plot({"data": [fig1],
#                      "layout": mylayout},
#                      auto_open=True,
#                      filename=("4DPlot.html"))

'4DPlot.html'